# <font color='blue'>Data Science Academy</font>
# <font color='blue'>Processamento de Linguagem Natural</font>


## Mini-Projeto 3
### Reconhecimento da Fala - Detectando Emoções em Arquivos de Áudio com Inteligência Artificial
### Parte 2 - Preparação dos Dados, Treinamento e Avaliação do Detector de Emoções com Deep Learning

Nota: Este jupyter notebook leva um bom tempo para ser executado. Se preferir, você pode apenas estudar este jupyter notebook e na Parte 3 o modelo será fornecido a você treinado e funcional.

In [ ]:
# Versão da Linguagem Python
from platform import python_version
print('Versão da Linguagem Python Usada Neste Jupyter Notebook:', python_version())

Este é um Mini-Projeto especial. 

Vamos trabalhar com uma das tarefas mais complexas em Inteligência Artificial: extrair emoções a partir da voz em arquivos de áudio.

Detectar emoções é uma das estratégias de Marketing mais importantes no mundo de hoje. Você pode personalizar aplicações para fornecer tratamentos diferentes para um indivíduo de acordo com a emoção detectada na voz de uma pessoa. Esse tipo de aplicação é um dos pilares para uma solução completa de IA.

Alguns exemplos de aplicações desse tipo de solução, incluem:

- Uma central de atendimento que toca músicas diferentes de acordo com a emoção detectada na voz do cliente. 

- Um carro autônomo que desacelera quando alguém está com raiva ou com medo. 

- Assistente pessoal que reage de acordo com a emoção detectada na voz.

- Aplicações de Marketing que oferecem diferentes produtos ou opções de acordo com a emoção do cliente.

- Assistente Virtual, que pode ser uma Enfermeira Virtual ou mesmo um Professor de Inglês, e que reage de acordo com a voz do interlocutor.

Entre outros exemplos.

Usaremos a biblioteca Librosa em Python para processar e extrair recursos dos arquivos de áudio. Librosa é um pacote Python para análise de música e áudio. Ele fornece os componentes necessários para criar sistemas de recuperação de informações musicais. 

Usando a biblioteca librosa, conseguimos extrair recursos através do MFCC (Mel Frequency Cepstral Coefficient). Os MFCCs são coeficientes amplamente usado no reconhecimento automático de fala. 

Também separamos a voz de mulheres e homens usando os identificadores fornecidos no dataset, como forma de deixar o modelo de reconhecimento de voz ainda mais preciso e personalizado.

Cada arquivo de áudio fornece muitos recursos, que são basicamente uma matriz de muitos valores. A esses recursos, iremos atribuir rótulos especificando o gênero da voz no áudio e a emoção detectada.

Como este projeto é um grande esforço de trabalho, ele foi dividido em 3 partes:

- **Parte 1 - Preparação dos Dados, Treinamento e Avaliação do Detector de Emoções com Machine Learning**
- **Parte 2 - Preparação dos Dados, Treinamento e Avaliação do Detector de Emoções com Deep Learning**
- **Parte 3 - Detecção e Classificação de Emoções em Arquivos de Áudio**

Este projeto pode ser facilmente adaptado aos seus próprios projetos. Tudo que você precisa é providenciar arquivos de áudio gravados com pessoas com diferentes emoções. Usaremos um dataset público para nosso trabalho.

Esta é a Parte 2. Aprenda e divirta-se.

In [ ]:
# Para atualizar um pacote, execute o comando abaixo no terminal ou prompt de comando:
# pip install -U nome_pacote

# Para instalar a versão exata de um pacote, execute o comando abaixo no terminal ou prompt de comando:
# pip install nome_pacote==versão_desejada

# Depois de instalar ou atualizar o pacote, reinicie o jupyter notebook.

# Instala o pacote watermark. 
# Esse pacote é usado para gravar as versões de outros pacotes usados neste jupyter notebook.
!pip install -q -U watermark

In [ ]:
# Instala pacote librosa
!pip install -q librosa

In [ ]:
# Imports
import os
import sys
import glob
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import librosa
import librosa.display
import scipy.io.wavfile
import tensorflow as tf
import keras
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from matplotlib.pyplot import specgram
from keras.models import Sequential, Model
from keras.layers import Dense, Embedding, Conv1D, MaxPooling1D, AveragePooling1D, BatchNormalization, Input, Flatten, Dropout, Activation
from keras.utils import to_categorical, np_utils

In [ ]:
# Versões dos pacotes usados neste jupyter notebook
%reload_ext watermark
%watermark -a "Data Science Academy" --iversions

Vamos criar um classe para o plot durante o treinamento.

In [ ]:
# Classe para o plot durante o treinamento
class LivePlot(keras.callbacks.Callback):
    
    # Função usada no começo do treinamento
    def on_train_begin(self, logs={}):
        self.losses = []
        self.acc = []
        self.val_losses = []
        self.val_acc = []
        self.logs = []
    
    # Função usada no fim do treinamento
    def on_epoch_end(self, epoch, logs={}):
        self.logs.append(logs)
        self.losses.append(logs.get('loss'))
        self.acc.append(logs.get('acc'))
        self.val_losses.append(logs.get('val_loss'))
        self.val_acc.append(logs.get('val_acc'))
        
        # Antes de plotar, certifique-se de que pelo menos 2 épocas tenham passado
        if len(self.losses) > 1:
            
            clear_output(wait=True)
            N = np.arange(0, len(self.losses))
            plt.style.use("seaborn")
            
            # Plot 
            plt.figure()
            plt.plot(N, self.losses, label = "train_loss")
            plt.plot(N, self.acc, label = "train_acc")
            plt.plot(N, self.val_losses, label = "val_loss")
            plt.plot(N, self.val_acc, label = "val_acc")
            plt.title("Acurácia e Erro de Treinamento na [Epoch {}]".format(epoch))
            plt.xlabel("Epoch #")
            plt.ylabel("Acurácia e Erro")
            plt.legend()
            plt.show()

A classe abaixo será usada para mapear os sentimentos.

In [ ]:
# Classe para mapear os sentimentos
class FetchLabel():

    def get_emotion(self, file_path):
        item = file_path.split('/')[-1]
        if item[6:-16]=='02' and int(item[18:-4])%2==0:
            return 'female_calm'
        elif item[6:-16]=='02' and int(item[18:-4])%2==1:
            return 'male_calm'
        elif item[6:-16]=='03' and int(item[18:-4])%2==0:
            return 'female_happy'
        elif item[6:-16]=='03' and int(item[18:-4])%2==1:
            return 'male_happy'
        elif item[6:-16]=='04' and int(item[18:-4])%2==0:
            return 'female_sad'
        elif item[6:-16]=='04' and int(item[18:-4])%2==1:
            return 'male_sad'
        elif item[6:-16]=='05' and int(item[18:-4])%2==0:
            return 'female_angry'
        elif item[6:-16]=='05' and int(item[18:-4])%2==1:
            return 'male_angry'
        elif item[6:-16]=='06' and int(item[18:-4])%2==0:
            return 'female_fearful'
        elif item[6:-16]=='06' and int(item[18:-4])%2==1:
            return 'male_fearful'
        elif item[6:-16]=='01' and int(item[18:-4])%2==0:
            return 'female_neutral'
        elif item[6:-16]=='01' and int(item[18:-4])%2==1:
            return 'male_neutral'
        elif item[6:-16]=='07' and int(item[18:-4])%2==0:
            return 'female_disgusted'
        elif item[6:-16]=='07' and int(item[18:-4])%2==1:
            return 'male_disgusted'
        elif item[6:-16]=='08' and int(item[18:-4])%2==0:
            return 'female_surprised'
        elif item[6:-16]=='08' and int(item[18:-4])%2==1:
            return 'male_surprised'
        elif item[:1]=='a':
            return 'male_angry'
        elif item[:1]=='f':
            return 'male_fearful'
        elif item[:1]=='h':
            return 'male_happy'
        elif item[:1]=='n':
            return 'male_neutral'
        elif item[:2]=='sa':
            return 'male_sad'
        elif item[:1]=='d':
            return 'male_disgusted'
        elif item[:2]=='su':
            return 'male_surprised'

## Leitura dos Arquivos de Áudio

Os arquivos de áudio estão no Titan, na pasta /media/datasets/PLN/Cap12/Tarefa4/.

In [ ]:
# Lista para os arquivos
mylist = []

In [ ]:
# Loop pela pasta e extrai o nome de cada arquivo
for path, subdirs, files in os.walk('/media/datasets/PLN/Cap12/Tarefa4/dados'):
    for name in files:
        mylist.append(os.path.join(path, name))

In [ ]:
mylist

## Plot dos Arquivos de Áudio

In [ ]:
# Extrai o sampling_rate dos arquivos
data, sampling_rate = librosa.load(mylist)
plt.figure(figsize=(15, 5))
librosa.display.waveplot(data, sr=sampling_rate)

In [ ]:
# Leitura dos arquivos
sr,x = scipy.io.wavfile.read(mylist)
nstep = int(sr * 0.01)
nwin  = int(sr * 0.03)
nfft = nwin
window = np.hamming(nwin)
nn = range(nwin, len(x), nstep)
X = np.zeros( (len(nn), nfft//2) )

In [ ]:
# Plot
for i,n in enumerate(nn):
    xseg = x[n-nwin:n]
    z = np.fft.fft(window * xseg, nfft)
    X[i,:] = np.log(np.abs(z[:nfft//2]))

plt.imshow(X.T, interpolation='nearest',
    origin='lower',
    aspect='auto')
plt.show()

## Extração de Features dos Arquivos de Áudio

In [ ]:
# Cria o dataframe
label = fetch_label.FetchLabel()
df = pd.DataFrame(columns = ['feature','emotion'])
bookmark=0

In [ ]:
# Loop para extração das features
for index,y in enumerate(mylist):
    X, sample_rate = librosa.load(y, res_type = 'kaiser_fast', duration = 3, sr = 22050*2, offset = 0.5)
    sample_rate = np.array(sample_rate)
    mfccs = np.mean(librosa.feature.mfcc(y=X, sr=sample_rate, n_mfcc=13), axis=0)
    feature = mfccs
    emotion = label.get_emotion(y)
    df.loc[bookmark] = [feature, emotion]
    bookmark=bookmark+1        

## Preprocessamento dos Dados

In [ ]:
feature_df = pd.DataFrame(df['feature'].values.tolist())
labelled_df = pd.concat([feature_df,df['emotion']], axis=1)
df_cleaned = labelled_df.dropna(0)
shuffled_df = df_cleaned.sample(frac=1).reset_index(drop=True)
shuffled_df = shuffled_df.loc[~shuffled_df['emotion'].isin(['male_neutral', 
                                                            'male_disgusted', 
                                                            'male_surprised', 
                                                            'female_neutral', 
                                                            'female_disgusted', 
                                                            'female_surprised'])]

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(shuffled_df.drop('emotion', axis = 1), 
                                                    shuffled_df['emotion'], 
                                                    test_size = 0.2, 
                                                    random_state = 0, 
                                                    shuffle = True)

## Codificando Rótulos

In [ ]:
lb = LabelEncoder()
y_train = np_utils.to_categorical(lb.fit_transform(y_train))
y_test = np_utils.to_categorical(lb.fit_transform(y_test))
np.save('labels/label_classes.npy', lb.classes_)

## Expansão de Dimensão

In [ ]:
x_train_exp = np.expand_dims(X_train, axis=2)
x_test_exp = np.expand_dims(X_test, axis=2)

## Construção e Treinamento do Modelo

In [ ]:
# Cria o modelo
model = Sequential()
model.add(Conv1D(256, 5, padding = 'same', input_shape = (x_train_exp.shape[1],1)))
model.add(Activation('relu'))
model.add(Conv1D(128, 5, padding = 'same'))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(MaxPooling1D(pool_size = (8)))
model.add(Conv1D(128, 5, padding = 'same',))
model.add(Activation('relu'))
model.add(Conv1D(128, 5, padding = 'same',))
model.add(Activation('relu'))
model.add(Flatten())
model.add(Dense(10))
model.add(Activation('softmax'))
opt = keras.optimizers.rmsprop(lr = 0.0001, decay = 1e-6)

In [ ]:
model.compile(loss = 'categorical_crossentropy', optimizer = opt, metrics = ['accuracy'])

In [ ]:
history = model.fit(x_train_exp, 
                    y_train, 
                    batch_size = 512, 
                    epochs = 600, 
                    validation_data = [x_test_exp, y_test], 
                    callbacks = [plot_losses])

## Avaliação do Modelo

In [ ]:
train_result = model.evaluate(x_train_exp, y_train, verbose=0)
test_result = model.evaluate(x_test_exp, y_test, verbose=0)

In [ ]:
print("train acc","%s: %.2f%%" % (model.metrics_names[1], train_result[1]*100))
print("train acc","%s: %.2f%%" % (model.metrics_names[1], test_result[1]*100))

In [ ]:
preds = model.predict(x_test_exp, batch_size=512, verbose=1)
preds_mod = preds.argmax(axis=1)
preds_flat = preds_mod.astype(int).flatten()
preds_transformed = (lb.inverse_transform((preds_flat)))

In [ ]:
pred_df = pd.DataFrame({'predictedvalues': preds_transformed})
actual_values = y_test.argmax(axis=1)
actual_values_mod = actual_values.astype(int).flatten()
actual_values_mod_transformed = (lb.inverse_transform((actual_values_mod)))
actual_df = pd.DataFrame({'actualvalues': actual_values_mod_transformed})
final_df = actual_df.join(pred_df)
final_df.head(10)

In [ ]:
final_df.groupby('actualvalues').count()

In [ ]:
final_df.groupby('predictedvalues').count()

In [ ]:
model_name = 'detecta_emotion.h5'
save_dir = os.path.join(os.getcwd(), 'modelos')

In [ ]:
# Salva o modelo
if not os.path.isdir(save_dir):
    os.makedirs(save_dir)
model_path = os.path.join(save_dir, model_name)
model.save(model_path)
print('Modelo salvo em %s ' % model_path)

Na Parte 3 usaremos o modelo treinado para detectar a emoção em arquivos de áudio.

# Fim